Case 2b: Nickel strip with circular cut profile but with center of circles out of the strips

With the center of the circle not constrained to be within the strips, this opens up an extra dimension of solutions to us, albeit having it come with an extra variable to tune. The profile is as such:

![](IMG_0746.jpg)
![](IMG_0747.jpg)

Import necessary libraries:

In [1]:
import numpy as np
from math import pi, sqrt, asin
import matplotlib.pyplot as plt

Define constants

In [37]:
I = 50.0            # current passing through the strip [A]
rho_d = 8890.0e-9   # density of nickel [kg/mm^3]
rho_r = 0.096e-3    # electrical resistivity of nickel [Ohm*mm]
dt = 1.0            # elapsed time [s]
h = 0.2             # nickel strip thickness [mm]
w0 = 8              # nickel strip default width [mm]
cp = 465            # sp. heat capacity of nickel [J/kgK]
Tm = 1435           # melting point temperature of nickel [C]
Tamb = 20           # ambient temperature [C]
Lf = 293e3          # sp. latent heat of nickel [J/kg]

With the local and global axes placed differently, we will have to adjust our original circle equation:

$$X^2 + Y^2 = r^2$$
$$X = x, Y = y + d$$
$$\therefore x^2 + (y + d)^2 = r^2$$
$$y(x,r,d) = \sqrt{r^2 - x^2} - d,\hspace{1cm} x \in \left[-\sqrt{r^2 - d^2}, \sqrt{r^2 - d^2}\right]$$
<br>

$$f(x,r,d) = w_0 - 2y = w_0 + 2d - 2\sqrt{r^2 - x^2},\hspace{1cm} x \in \left[-\sqrt{r^2 - d^2}, \sqrt{r^2 - d^2}\right]$$ 

Where $y$ is the height of the segment of the circle on the strip. At the end of the day, we solve for $d$ for each cases of $r$. From the above equation, it is easily seen that the maximum segment height, $y_m$, is equal to:

$$y_m = y(0,r,d) = r - d$$

Furthermore, $d$ have to respect these bounds:

$$2y_m \leq w_0$$
$$2(r - d) \leq w_0$$
$$d \geq \frac{2r - w_0}{2}$$ 
<br>

$$r^2 - d^2 \geq 0$$
$$d \leq r$$ 
<br>

$$\therefore d \in \left[\frac{2r - w_0}{2}, r\right]





In [2]:
# define functions f(x,r,d) and g(x,r,d)

def f(x,r,d): 
    if r**2 - x**2 < 1e-15:
        return w0 + 2*d
    else: return w0 + 2*d - 2*sqrt(r**2 - x**2)

def g(x,r,d): return 1/f(x,r,d)

In [4]:
# define functions of lower bound and upper bounds, x0(r,d) and xf(r,d)

def xf(r,d):
    if r**2 - d**2 < 1e-15:
        return 0
    else: return sqrt(r**2 - d**2)

def x0(r,d): return -xf(r,d)

As with case 2a, we define:

$$\int_{x_0}^{x_f}{g(x,r,d) dx} \approx G_n(r,d) =  \frac{x_f - x_0}{3}\left(g(x_0,r,d)+g(x_f,r,d)+2\sum_{i = 1}^{n-2}g(x_{2i},r,d)+4\sum_{i = 1}^{n-1}g(x_{2i-1},r,d)\right)$$

In [32]:
# define simpsons rule to carry out numerical integration
# size MUST be an odd number
# **vars are all additional arguments needed to evaluate the passed function

def simpsons(function,size,lower_bound,upper_bound, **vars):
    delta_x = (upper_bound - lower_bound)/(size - 1)
    x = np.linspace(lower_bound,upper_bound,size)
    points = (function(x, **vars))

    result = (delta_x/3)*(points[0] + points[-1] + 2*sum(points[2:size - 2:2]) + 4*sum(points[1:size - 1:2]))
    return result

In [31]:
a = 0
b = pi
n = 11
x = np.linspace(a,b,n)
def k(x,r): return np.sin(x*r)

I_simp = simpsons(k,n,a,b, r = 2)
print(I_simp)


4.4108427377215436e-17


To define the volume, we setup our geometry as such:

![](IMG_0748.jpg)

We can find out the angle $\theta$ using the identity:
$$\sin{\theta} = \frac{x_f}{r}$$
$$\therefore \theta = \arcsin{\frac{\sqrt{r^2 - d^2}}{r}}$$

So, the area of the segment is:
$$A = 2\left(\frac{r^2}{2}\arcsin{\left(\frac{\sqrt{r^2 - d^2}}{r}\right)} - \frac{d}{2}\sqrt{r^2 - d^2}\right) = r^2\arcsin{\left(\frac{\sqrt{r^2 - d^2}}{r}\right)} - d\sqrt{r^2 - d^2}$$

The volume is then:
$$V(r,d) = hA_{\text{face}} = h(w_0l - 2A) = 2(w_0h + d)\sqrt{r^2 - d^2} - 2r^2h\arcsin{\left(\frac{\sqrt{r^2 - d^2}}{r}\right)}$$

For convenience, as done in case 2a, we leave the $h$ out of the volume function and lump it with the other constants. Redefine:

$$V(r,d) = A_{\text{face}} = (w_0l - 2A) = 2(w_0 + d)\sqrt{r^2 - d^2} - 2r^2\arcsin{\left(\frac{\sqrt{r^2 - d^2}}{r}\right)}$$
$$K = \frac{\rho_dh^2}{I^2\rho_r\Delta{t}}\left(c_p\left(T_m - T_\infty\right) + L_f\right)$$

Let:

$$\frac{G_n(r,d)}{V(r,d)} = H_n(r,d)$$

In [34]:
# define function of volume, V(r,d)
def V(r,d): return 2*(w0 + d)*sqrt(r**2 - d**2) - 2*r**2*asin(sqrt(r**2 - d**2)/r)

# define lumped function H(r,d)
def H(function,size,lower_bound,upper_bound, **vars): return simpsons(function,size,lower_bound,upper_bound, **vars)/V(**vars)
    

In [35]:
# define the upper and lower limits for d, ul(r) and ll(r)

def ll(r): return (2*r - w0)/2
def ul(r): return r

In [38]:
# the lumped constant
K = rho_d*h**2*(cp*(Tm - Tamb) + Lf)/(I**2*rho_r*dt)

We now build a convergence algorithm for each tested cases of r:

In [40]:
def find_zero(function,eps,init_value,alpha, **vars):
    error = 1
    k = 1
    result = init_value

    while abs(error) > eps:
        error = function(**vars)
        result -= alpha/k*error
        k += 1

    return result

In [ ]:
R = 